In [4]:
import sys

python_path = ['/home/claudio/.local/lib64/python3.6/site-packages','/home/sebastian/.local/lib64/python3.6/site-packages','/home/sebastian/.local/lib/python3.6/site-packages','/home/sebastian/.local/lib/python3.6/site-packages/opencv-4.1.0-py3.6.egg/cv2',
               '/home/sebastian/.local/lib/python3.6/site-packages/cv2','/home/sebastian/dat/onnx/onnx-tensorrt/third_party/onnx']
sys.path.extend(python_path)
from torchnlp.word_to_vector import FastText
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
from utils import *
from zero_shot_model import *
from PIL import Image
from PIL import ImageFile

In [435]:
data = get_zeroshot_dataset(True)

['ballroom', 'bar_lounge', 'basics', 'bathroom', 'beach', 'breakfast', 'exterior_view', 'golf_course', 'guest_room', 'interior_view', 'living_room', 'lobby_view', 'meeting_room', 'pool_view', 'property_amenities', 'recreational_facility', 'spa']


/home/claudio/notebooks/claudio/zero-shot-learning/utils.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['file'] = tags['file']


In [436]:
test_data = get_zeroshot_dataset(False)

['ballroom', 'bar_lounge', 'basics', 'bathroom', 'beach', 'breakfast', 'exterior_view', 'golf_course', 'guest_room', 'interior_view', 'living_room', 'lobby_view', 'meeting_room', 'pool_view', 'property_amenities', 'recreational_facility', 'spa']


/home/claudio/notebooks/claudio/zero-shot-learning/utils.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['file'] = tags['file']


In [437]:
dataset = ZeroshotDataset('/data/hotel_images/', data)

In [438]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

In [439]:
def embedding_sofmax_loss(projections, labels, embedding_matrix):
    
    sum_logits = torch.zeros(projections.shape[1], embedding_matrix.shape[1]).to(device)
    
        
    for i in range(4):    
        sum_logits += torch.mm(projections[i], embedding_matrix)
        
    target = torch.topk(labels.long(), k=1, dim=1)[1]

    
    return criterion(sum_logits, target.T[0])

In [154]:
def class_center_triplet_loss(centers, labels, projections, margin=0.8):
    
    p = torch.sum(projections, dim=0)
    
    p = p / p.norm(p=2, dim=1).view(-1,1)
    
    c = centers / (centers.norm(p=2, dim=1)).view(-1,1)
    
    one_positions = torch.argmax(labels, dim=1)
    
    print(one_positions.shape)
    
    center_distances = torch.empty(p.shape[0], centers.shape[0])
    for i in range(centers.shape[0]):
        center_distances[:, i] = (p - centers[i]).norm(p=2, dim=1)**2
        
    total_loss = torch.FloatTensor([0])
    for i in range(center_distances.shape[0]):
        sample_distances = center_distances[i]
        idx = one_positions[i]
        center_class_distance = sample_distances[idx]
        
        sample_loss = torch.FloatTensor([0])
        for j in range(sample_distances.shape[0]):
            if j != idx:
                sample_loss += torch.max(torch.FloatTensor([0]), margin + center_class_distance - sample_distances[j])
    
        total_loss += sample_loss
        
    return total_loss / projections.shape[0]
    

In [157]:
centers = torch.randn(20, 512, requires_grad=True)
labels = torch.eye(20)
projections = torch.randn(4,20,512)

In [158]:
class_center_triplet_loss(centers, labels, projections)

torch.Size([20])


tensor([1579.4961], grad_fn=<DivBackward0>)

In [451]:
optimizer = torch.optim.Adam(model.get_model_parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-06, weight_decay=0, amsgrad=False)

In [57]:
epochs = 5

for epoch in range(epochs):
    
    for images, labels in dataloader:
        
        optimizer.zero_grad()
        
        projections, multi_attention_loss, centers = model(images.to(device))
        
        loss = embedding_sofmax_loss(projections, labels.to(device), embedding_matrix) + multi_attention_loss
        
        loss.backward()
        
        optimizer.step()
        
        model.eval_mode()
        
        with torch.no_grad():
        
            for images, labels in test_data:
                
                projections = model(images.to(device))
                
                logits = torch.mm(projections, embedding_matrix)
                
                pred, indices_pred = torch.topk(1, logits)
                
                indices_true = torch.topk(1, labels.to(device))
                
                print(indices_true.shape)
                print(indices_pred.shape)
            
            
    

/home/claudio/.local/lib/python3.6/site-packages/torch/nn/functional.py:2705: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 11.91 GiB total capacity; 10.59 GiB already allocated; 601.06 MiB free; 10.75 GiB reserved in total by PyTorch)

In [118]:
device = 'cuda:0'

In [119]:
model = ZeroshotModel.load_model(device, 'models/zeroshot_model_0.91/')

In [120]:
batch_size = 16

embedding_matrix = get_category_embedding_matrix().to(device)

train_data = get_zeroshot_dataset(True)

test_data = get_zeroshot_dataset(False)

train_dataset = ZeroshotDataset('/data/hotel_images/', train_data)

train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle=False)

test_dataset = ZeroshotDataset('/data/hotel_images/', test_data)

test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

['ballroom', 'bar_lounge', 'basics', 'bathroom', 'beach', 'breakfast', 'exterior_view', 'golf_course', 'guest_room', 'interior_view', 'living_room', 'lobby_view', 'meeting_room', 'pool_view', 'property_amenities', 'recreational_facility', 'spa']


/home/claudio/notebooks/claudio/zero-shot-learning/utils.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['file'] = tags['file']


['ballroom', 'bar_lounge', 'basics', 'bathroom', 'beach', 'breakfast', 'exterior_view', 'golf_course', 'guest_room', 'interior_view', 'living_room', 'lobby_view', 'meeting_room', 'pool_view', 'property_amenities', 'recreational_facility', 'spa']


/home/claudio/notebooks/claudio/zero-shot-learning/utils.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['file'] = tags['file']


In [21]:
model.eval_mode()
        
with torch.no_grad():

    acc = 0
    for images, labels in test_dataloader:

        projections, _ = model(images.to(device))

        sum_logits = torch.zeros(projections.shape[1], embedding_matrix.shape[1]).to(device)


        for i in range(4):
            sum_logits += torch.mm(projections[i], embedding_matrix)


        sum_logits = torch.nn.functional.softmax(sum_logits, dim = 1)

        indices_pred = torch.topk(sum_logits, k = 2, dim = 1)[1].detach().cpu().numpy()

        indices_true = torch.topk(labels.to(device), k = 2)[1].detach().cpu().numpy()


        for i in range(indices_true.shape[0]):
            intersection = set(indices_pred[i].tolist()).intersection(indices_true[i].tolist())
            acc += 1 if len(intersection) > 0 else 0


    acc /= len(test_dataset)

    print(acc)

/home/claudio/.local/lib/python3.6/site-packages/torch/nn/functional.py:2705: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


0.9362745098039216


In [6]:
import torch

In [16]:
a = np.empty((3,3))
a.fill(2)

In [17]:
a = torch.from_numpy(a)

In [18]:
a

tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], dtype=torch.float64)

In [19]:
norm = a.norm(p=2, dim=1)

In [20]:
norm

tensor([3.4641, 3.4641, 3.4641], dtype=torch.float64)

In [22]:
c = a / norm

In [23]:
c.norm(p=2, dim=1)

tensor([1., 1., 1.], dtype=torch.float64)

In [24]:
i = torch.eye(3)

In [30]:
indices = torch.tensor([[1],[1],[1]])

In [32]:
i[indices]

tensor([[[0., 1., 0.]],

        [[0., 1., 0.]],

        [[0., 1., 0.]]])

In [33]:
i - i[0]

tensor([[ 0.,  0.,  0.],
        [-1.,  1.,  0.],
        [-1.,  0.,  1.]])

In [34]:
len(i)

3

In [37]:
0 + i[0][0]

tensor(1.)